In [ ]:
X_train = x_train.reshape(50000, -1)
X_test = x_test.reshape(10000, -1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# normalization - converting ints to doubles
X_train /= 255
X_test /= 255
X_train.shape, X_test.shape

In [ ]:
n_inputs = 3072  # Each image now is 3072 long  
n_hidden1 = 400  # Num of nodes in hidden layer 1
n_hidden2 = 250  # Num of nodes in hidden layer 2     
n_hidden3 = 64   # Num of nodes in hidden layer 3
n_outputs = 10   # number of classes

In [ ]:
X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X") 
y = tf.placeholder(tf.int32, shape=(None), name = "y")
training = tf.placeholder_with_default(False, shape=(), name='training')

In [ ]:
from functools import partial  

batch_norm_momentum = 0.9
with tf.name_scope("dnn"):
    
    he_init = tf.contrib.layers.xavier_initializer()
    
    # 3 Hidden layers, each using sigmoid activation function

    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
    bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9) 
    # Momentum of 0.9 is best known for optimization.
    bn1Activation = tf.nn.sigmoid(bn1)
    
    hidden2 = tf.layers.dense(bn1Activation, n_hidden2, name = "hidden2")
    bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
    bn2Activation = tf.nn.sigmoid(bn2)

    hidden3 = tf.layers.dense(bn2Activation, n_hidden3, name = "hidden3")
    bn3 = tf.layers.batch_normalization(hidden3, training = training, momentum = 0.9)
    bn3Activation = tf.nn.sigmoid(bn3)
    logits_before_bn = tf.layers.dense(bn3Activation, n_outputs, name = "outputs") 
    

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits =logits_before_bn)
    # labels - vector must provide a single specific index for the true class
    
    loss = tf.reduce_mean(xentropy, name="loss")
    

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.05, epsilon=1e-06, use_locking=True,name='Adam')
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits_before_bn, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [ ]:
init = tf.global_variables_initializer()      # initializes the tf.variable/constant/placeholders declared above
saver = tf.train.Saver()                      # helps to save the model and reuse later.

In [ ]:
# Now divide the test and train datasets into 3 parts, training set, validation set, and testing set
# Considering the validation set to have 5000 images
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [ ]:
n_epochs = 25
batch_size = 6000 

# A function to shuffle the whole data set and select different batches randomly with replacement
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch                         # returns the first batch that was formed

In [ ]:
# Now lets test the model with the built model on Validation Set

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch.reshape([-1])})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid.reshape([-1])})
        print(epoch, "Validation accuracy:", accuracy_val)

In [ ]:
# test the results on test set

n_epochs = 25
batch_size = 5000

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch.reshape([-1])})
        if epoch % 2 == 0:
            accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape([-1])})
            print(epoch, "TestSet accuracy:", accuracy_val)

In [ ]:
bn1Activation = tf.nn.relu(bn1)
bn2Activation = tf.nn.relu(bn2)
bn3Activation = tf.nn.relu(bn3)

In [ ]:
# Now lets test the model with the built model on Validation Set

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch.reshape([-1])})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid.reshape([-1])})
        print(epoch, "Validation accuracy:", accuracy_val)

In [ ]:
# test the results on test set

n_epochs = 25
batch_size = 5000

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch.reshape([-1])})
        if epoch % 2 == 0:
            accuracy_val = accuracy.eval(feed_dict={X: X_test, y: y_test.reshape([-1])})
            print(epoch, "TestSet accuracy:", accuracy_val)